# Data Prep for CPS: Robotics
Nathan Butler  |  nlbutler@iastate.edu

Following are some instructions for all the theme groups.
1. The final code must be pushed to git before the deadline.
2. Use the discussion board in Canvas in case of any issue.

If the group has technical questions related to the homework, don’t hesitate to get in touch with Ryan (hjy@iastate.edu)

----
In robotics application, we often have to pre-process the input images being used by a robot to improve the learning capabilities of the robot. We will explore a few examples of such pre-processing techniques in this assignment.

We will use the Grasping Dataset available at  https://www.kaggle.com/oneoneliu/cornell-grasp to pre-process images so that a robot arm can learn to detect and grasp an object.

The dataset contains raw images of the objects, coordinates and labels of grasping rectangles as well as point cloud data of the objects. 

# Part 1
Download the dataset and overlay the positive and negative grasping rectangles for each image on the raw image using the coordinates of the rectangles provided.

In [ ]:
#Generated by ChatGPT:

# Imports
import os
import urllib.request
import zipfile
import numpy as np
from PIL import Image, ImageDraw

# Toggle download of dataset (only run once)
download = True

if download:
    # Download dataset from Kaggle
    url = 'https://www.kaggle.com/oneoneliu/cornell-grasp'
    os.environ['KAGGLE_USERNAME'] = '<your_kaggle_username>'
    os.environ['KAGGLE_KEY'] = '<your_kaggle_key>'
    !kaggle datasets download -d oneoneliu/cornell-grasp
    with zipfile.ZipFile('cornell-grasp.zip', 'r') as zip_ref:
        zip_ref.extractall()

# Load dataset
dataset_path = './cornell-grasp'
image_dir = os.path.join(dataset_path, 'cornell_dataset', 'rgb')
pos_dir = os.path.join(dataset_path, 'cornell_dataset', 'pos')
neg_dir = os.path.join(dataset_path, 'cornell_dataset', 'neg')

image_filenames = sorted(os.listdir(image_dir))

# Overlay grasping rectangles on images
for filename in image_filenames:
    # Load image and grasping rectangles
    img = Image.open(os.path.join(image_dir, filename))
    pos_rects = np.loadtxt(os.path.join(pos_dir, filename.replace('.jpg', '.txt')))
    neg_rects = np.loadtxt(os.path.join(neg_dir, filename.replace('.jpg', '.txt')))
    
    # Draw grasping rectangles on image
    draw = ImageDraw.Draw(img)
    for rect in pos_rects:
        draw.rectangle(rect, outline='green', width=3)
    for rect in neg_rects:
        draw.rectangle(rect, outline='red', width=3)
    
    # Show image
    img.show()